This notebook makes RapidAPI calls to the Botometer endpoint.
See the [Botometer Repository](https://github.com/IUNetSci/botometer-python) for documentation on the requirements to make Botometer requests.

In [ ]:
import os
import pandas as pd
import botometer
import csv
#botometer requires Tweepy version 3.10 or lower
import tweepy
import config

consumer_key = config.CONSUMER_KEY
consumer_secret = config.CONSUMER_SECRET
access_token = config.ACCESS_TOKEN
access_token_secret = config.ACCESS_TOKEN_SECRET
rapidapi_key = config.RAPIDAPI_KEY

In [ ]:
input_name = "./data_csv/intermediate/stream_users.csv"
output_name = "./data_csv/final/stream_account_scores.csv"

assert not os.path.exists(output_name), f"{output_name} already exists"

df = pd.read_csv(input_name, names=range(14))

names = df[4]

twitter_app_auth = {
    'consumer_key': consumer_key,
    'consumer_secret': consumer_secret,
    'access_token': access_token,
    'access_token_secret': access_token_secret,
  }
bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)


with open(output_name, "w") as csv_file:
    writer = csv.writer(csv_file, delimiter=',')

    for name in names:
        try:
            print(name)
            result = bom.check_account(name)
            threshold = result["cap"]["universal"]
            raw_overall = result["raw_scores"]["universal"]["overall"]

            writer.writerow([name, threshold, raw_overall])
        
        except tweepy.TweepError as e:
            print(e)